# PLAN
## Initial
#### Safe bet to use Alpha Vantage to develop Model
## Final Step
#### Once ready to do extensive tranning with Multiple Stocks we will need to use SEC EDGAR most likely (need to figure that out) or pay for like a month subscription of Alpha Vantage

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta


# Using FMP
# - This is more accurate and consitent! Yahoo is web scrapping
# - Provides the financial statetement for a broader range then yahoo finance

# Key which is given from my FMP account. (This key is needed to make requests) Maximum of 200 calls/day
API_KEY = "wUvM2M29ZVxHDvK8IRp2P7iyrT8uhQG4"

ticker = "AAPL"

end_date = datetime(2024, 6, 1)
start_date = end_date - timedelta(days=5000)

url_hist_stock_price = f"https://financialmodelingprep.com/stable/historical-price-eod/light?symbol={ticker}&from={start_date.date()}&to={end_date.date()}&apikey={API_KEY}"

# Historical Data Request
response_hist_stock_price = requests.get(url_hist_stock_price)

json_response_hist_stock_price = response_hist_stock_price.json()


In [2]:
closing_price = pd.DataFrame(json_response_hist_stock_price)

# Convert 'date' to datetime type
closing_price['date'] = pd.to_datetime(closing_price['date'])

# Keep only 'date' and 'price' columns
closing_price = closing_price[['date', 'price']]

closing_price['price'] = pd.to_numeric(closing_price['price'], errors='coerce')
closing_price['date'] = pd.to_datetime(closing_price['date'], errors='coerce')
# Set 'date' as the index
closing_price.set_index('date', inplace=True)

print(closing_price)

             price
date              
2024-05-31  192.25
2024-05-30  191.29
2024-05-29  190.29
2024-05-28  189.99
2024-05-24  189.98
...            ...
2010-09-29   10.26
2010-09-28   10.24
2010-09-27   10.40
2010-09-24   10.44
2010-09-23   10.32

[3445 rows x 1 columns]


In [3]:
# ALPHA VANTAGE EXPERIMENTING (https://www.alphavantage.co/documentation/#)
# gives some insight into the different attributes without sending demo request: https://documentation.alphavantage.co/FundamentalDataDocs/gaap_documentation.html

API_KEY = "FJLURMBXQEXWW9PP"

ticker = "AAPL"

def getInfo(function:str,ticker:str):
    url = f"https://www.alphavantage.co/query?function={function.upper()}&symbol={ticker}&apikey={API_KEY}"
    response = requests.get(url)
    return response.json()

In [4]:
# Requests
r_income_statements = getInfo("income_statement",ticker)
r_balance_sheet = getInfo("balance_sheet",ticker)
r_cash_flow = getInfo("cash_flow",ticker)

In [5]:
# Dataframes
df_income_statement = pd.DataFrame(r_income_statements["quarterlyReports"])
df_income_statement['fiscalDateEnding'] = pd.to_datetime(df_income_statement['fiscalDateEnding'])

#df_income_statement = df_income_statement.rename(columns={"fiscalDateEnding": "dates"})
df_income_statement.set_index("fiscalDateEnding", inplace=True)
df_income_statement = df_income_statement.apply(pd.to_numeric, errors='coerce')
df_income_statement = df_income_statement.fillna(0)



df_balance_sheet = pd.DataFrame(r_balance_sheet["quarterlyReports"])
df_balance_sheet['fiscalDateEnding'] = pd.to_datetime(df_balance_sheet['fiscalDateEnding'])
#df_balance_sheet = df_balance_sheet.rename(columns={"fiscalDateEnding": "dates"})
df_balance_sheet.set_index("fiscalDateEnding", inplace=True)
df_balance_sheet = df_balance_sheet.apply(pd.to_numeric, errors='coerce')
df_balance_sheet = df_balance_sheet.fillna(0)


df_cash_flow = pd.DataFrame(r_cash_flow["quarterlyReports"])
df_cash_flow['fiscalDateEnding'] = pd.to_datetime(df_cash_flow['fiscalDateEnding'])
#df_cash_flow = df_cash_flow.rename(columns={"fiscalDateEnding": "dates"})
df_cash_flow.set_index("fiscalDateEnding", inplace=True)
df_cash_flow = df_cash_flow.apply(pd.to_numeric, errors='coerce')
df_cash_flow = df_cash_flow.fillna(0)


In [6]:
# Available data
#print("Income Statement\n",df_income_statement.columns)
#print("Balance Sheet\n",df_balance_sheet.columns)
#print("Cash Flow\n",df_cash_flow.columns)

In [7]:
# TODO: Adjust all metrics such that they can be mapped to that of closing price (i.e the dates)

# Often used terms
net_income = df_income_statement['netIncome']
equity = df_balance_sheet["totalShareholderEquity"]
outstandingShares = df_balance_sheet["commonStockSharesOutstanding"]
book_value = equity
book_value_per_share = book_value/outstandingShares
total_revenue =df_income_statement['totalRevenue']

# Profitability Metrics
EPS = (net_income-df_cash_flow["dividendPayoutPreferredStock"])/outstandingShares
ROE = net_income/equity
net_Profit_Margin = net_income/total_revenue

# Valuation Metrics--> MOST PROBLEMATIC HERE
#P_E_Ratio = closing_price/EPS
#P_B_Ratio = closing_price/book_value_per_share
#P_S_Ratio = closing_price*outstandingShares/total_revenue

# Risk and Efficiency metrics
D_E_Ratio = df_balance_sheet["shortLongTermDebtTotal"]/equity
Current_Ratio = df_balance_sheet["totalCurrentAssets"]/df_balance_sheet["totalCurrentLiabilities"]



In [8]:
# INITIAL FEATURES

import pandas as pd
import numpy as np

columns_part1 = []
# Daily Return
daily_return = closing_price.pct_change()

# 5-Day Return
ret_5d = closing_price.pct_change(5)
# 10-Day Return
ret_10d = closing_price.pct_change(10)

# 5-Day Volatility (Std Dev of Returns)
vol_5d = closing_price.pct_change().rolling(window=5).std()

# 10-Day Volatility
vol_10d = closing_price.pct_change().rolling(window=10).std()

# Momentum (10d)
momentum_10d = closing_price - closing_price.shift(10)

# SMA_10/SMA_50 Ratio
sma_10 = closing_price.rolling(window=10).mean()
sma_50 = closing_price.rolling(window=50).mean()
sma_ratio = sma_10/sma_50

# Z-score (20d)
rolling_mean = closing_price.rolling(window=20).mean()
rolling_std = closing_price.rolling(window=20).std()
z_score_20d = (closing_price - rolling_mean)/rolling_std

# RSI (14d)
delta = closing_price.diff()
gain = delta.where(delta > 0, 0.0)
loss = -delta.where(delta < 0, 0.0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain/avg_loss
rsi_14 = 100 - (100 / (1 + rs))
y =  closing_price.pct_change(-1)*100
y.columns = ['y']

dataframes = [
    (closing_price,'price'),
    (daily_return, 'ret_1d'),
    (ret_5d, 'ret_5d'),
    (ret_10d, 'ret_10d'),
    (vol_5d, 'vol_5d'),
    (vol_10d, 'vol_10d'),
    (momentum_10d, 'momentum_10d'),
    (sma_ratio, 'sma_ratio_10_50'),
    (z_score_20d, 'zscore_20d'),
    (rsi_14, 'rsi_14'),
    (y,'y')
]

df_features_short_term = pd.concat(
    [df.rename(columns={df.columns[0]: name}) for df, name in dataframes],
    axis=1
)


In [9]:

#print(df_balance_sheet.columns)

outstandingShares_str = df_balance_sheet["commonStockSharesOutstanding"]
outstandingShares = outstandingShares_str.apply(pd.to_numeric, errors='coerce')
df_metrics = pd.DataFrame()

df_features_long_term = pd.concat([
    EPS.rename('EPS'),
    ROE.rename('ROE'),
    D_E_Ratio.rename('D_E_Ratio'),
    Current_Ratio.rename('Current_Ratio')

], axis=1)


combined_rows = []
quarter_dates = df_features_long_term.index
pos = 0

columns = ['date']
columns.extend(df_features_short_term.columns.to_list())
columns.extend(df_features_long_term.columns.to_list())
columns.extend(['P_E_ratio','P_B_ratio','P_S_ratio'])

for date,row in df_features_short_term.iterrows():
    index = quarter_dates[pos]
    if (pos < len(quarter_dates)):
        if index < date:
            new_row = [date]
            new_row.extend(row.to_list())
            new_row.extend(df_features_long_term.loc[index].to_list())

            bv_per_share_elem = book_value_per_share.loc[index]
            outstandingShares_elem = outstandingShares.loc[index]
            total_revenue_elem = total_revenue.loc[index]
            
            P_E_ratio = row.price/df_features_long_term.loc[index]['EPS']
            P_B_ratio = row.price/bv_per_share_elem
            P_S_ratio = row.price*outstandingShares_elem/total_revenue_elem

            new_row.extend([P_E_ratio,P_B_ratio,P_S_ratio])

            combined_rows.append(new_row)
        else:
            pos+=1
df_features = pd.DataFrame(combined_rows,columns=columns)
df_features = df_features.dropna()
df_features.set_index('date',inplace=True)
print(df_features.columns)

  # NEED TO COMBINE THE ROWS TOGHETHER INTO!



Index(['price', 'ret_1d', 'ret_5d', 'ret_10d', 'vol_5d', 'vol_10d',
       'momentum_10d', 'sma_ratio_10_50', 'zscore_20d', 'rsi_14', 'y', 'EPS',
       'ROE', 'D_E_Ratio', 'Current_Ratio', 'P_E_ratio', 'P_B_ratio',
       'P_S_ratio'],
      dtype='object')


In [10]:
# Train
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Features df
X = df_features.drop(columns=['y'])
# Target Columns
y = df_features['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Train a Linear Regression model (least squared)
model = LinearRegression()
model.fit(X_train, y_train) # Find A in y=Ax
print(model.score(X_test, y_test))

# Predict and evaluate
y_pred = model.predict(X_test) # Predict y_test using the A matrix found and X_test
mse = mean_squared_error(y_test,y_pred) # Compare the found y_test (y_prep) to the true y_test


-0.022836408252508544


In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42,n_estimators=400)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
q1 = y.quantile(0.25)
q3 = y.quantile(0.75)
iqr = q3 - q1
nRMSE = (mse*(1/2)) / iqr

print(nRMSE) # <0.5 excellent, 0.5-1.0 good, 1.0-1.5 moderate, >1.5 Weak

0.8011872707612097


In [12]:
result = pd.DataFrame()
result['y'] = y_test
result['y_pred'] = y_pred
print(result)

q1 = y.quantile(0.25)
q3 = y.quantile(0.75)
iqr = q3 - q1
nRMSE = (mse*(1/2)) / iqr
print(nRMSE) 

print('Mean Squared Error', mse)
print('Min of y:', q1)
print('Max of y:', q3)
print('error',nRMSE)

                   y    y_pred
date                          
2022-04-13  1.634260 -0.440039
2015-10-29  1.039571  0.433160
2021-05-17 -0.925853  0.204817
2018-11-05 -2.834008 -1.174417
2017-09-25 -0.869107  0.135394
...              ...       ...
2022-12-06 -2.536998  0.642214
2023-12-22 -0.554757  0.028026
2011-07-28 -0.213980 -0.298115
2013-08-02  1.287554  0.116895
2016-01-22  5.315615 -0.412675

[669 rows x 2 columns]
0.8011872707612097
Mean Squared Error 2.852520633643241
Min of y: -0.7487610428786873
Max of y: 1.031422403454063
error 0.8011872707612097


In [39]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up parameters for XGBoost
params = {
    'objective': 'reg:squarederror',  # Regression problem
    'max_depth': 6,                   # Depth of the tree
    'eta': 0.1,                       # Learning rate
    'subsample': 0.8,                 # Fraction of samples used for training
    'colsample_bytree': 0.8,          # Fraction of features used for training
    'eval_metric': 'rmse'             # Evaluation metric (Root Mean Squared Error)
}

# Train the model
num_round = 100  # Number of boosting rounds
model = xgb.train(params, dtrain, num_round)


In [44]:
y_pred = model.predict(dtest)

# Convert predictions to a DataFrame to see the results
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(predictions)

              Actual  Predicted
date                           
2022-04-13  1.634260  -0.106000
2015-10-29  1.039571  -0.120245
2021-05-17 -0.925853  -0.393181
2018-11-05 -2.834008  -1.151040
2017-09-25 -0.869107  -0.049455
...              ...        ...
2022-12-06 -2.536998   0.246484
2023-12-22 -0.554757   0.054814
2011-07-28 -0.213980  -0.187400
2013-08-02  1.287554  -0.734524
2016-01-22  5.315615   0.215521

[669 rows x 2 columns]


In [43]:
result = pd.DataFrame()
result['y'] = y_test
result['y_pred'] = y_pred
print(result)

q1 = y.quantile(0.25)
q3 = y.quantile(0.75)
iqr = q3 - q1
nRMSE = (mse*(1/2)) / iqr
print(nRMSE) 

print('Mean Squared Error', mse)
print('Min of y:', q1)
print('Max of y:', q3)
print('error',nRMSE)

                   y    y_pred
date                          
2022-04-13  1.634260 -0.106000
2015-10-29  1.039571 -0.120245
2021-05-17 -0.925853 -0.393181
2018-11-05 -2.834008 -1.151040
2017-09-25 -0.869107 -0.049455
...              ...       ...
2022-12-06 -2.536998  0.246484
2023-12-22 -0.554757  0.054814
2011-07-28 -0.213980 -0.187400
2013-08-02  1.287554 -0.734524
2016-01-22  5.315615  0.215521

[669 rows x 2 columns]
0.8011872707612097
Mean Squared Error 2.852520633643241
Min of y: -0.7487610428786873
Max of y: 1.031422403454063
error 0.8011872707612097
